# Python Tools for Monte Carlo: OpenMC and MontePy
## Introduction to MontePy
### Presented by: Travis Labossiere-Hickman<sup>1</sup>

### Prepared by: Travis Labossiere-Hickman, Micah Gale<sup>2</sup>,

Presented: April 27, 2025 at the International Conference on Mathematics and Computational Methods Applied to Nuclear Science and Engineering (M&C 2025)

<sup>1</sup> Idaho National Laboratory; University of Illinois, Urbana-Champaign; Contact: [Travis.LabossiereHickman@INL.gov](mailto:Travis.LabossiereHickman@INL.gov)

<sup>2</sup> Idaho National Laboratory; University of Wisconsin—Madison; Contact: [Micah.Gale@INL.gov](mailto:micah.gale@inl.gov)





# Table of Contents
* Introduction to MCNP
* Need for Automation
* Why MontePy?
* Introduction to MontePy documentation

# Introduction to MCNP
* (for those living under a rock)
* Monte Carlo N-Particle (MCNP) developed by Los Alamos National Lab
* Code dates to the 1960's<sup>1</sup>
* Input syntax predates XML, JSON, YAML, SGML, oh my
* Syntax developed for punchcards
  
  <img alt="a stack of punch cards" src="https://upload.wikimedia.org/wikipedia/commons/2/26/Punched_card_program_deck.agr.jpg" width="300"></img>

  (By Arnold Reinhold CC BY-SA 3.0)

### MCNP input files are made up of "inputs" not "cards"

<sup>1</sup> J. Kulesza et al., "MCNP Code Version 6.3.0 Theory & User Manual," LA-UR-22-30006, Rev. 1, September 2022.


```
Very important title
1 1 -19.81 -2 imp:n=1

2 so 10

m1 92235.80c 1.0
kcode 1000 1.5 50 200
ksrc 0 0 0
```


# Need for Automation
* We find these truths to be self-evident

## How many of you have done the following? 

### The lazy template

``` python
def update_radius(radius):
    with open("template.imcnp") as in_fh, open(f"new_{radius}.imcnp") as out_fh:
        for i, line in enumerate(in_fh):
            if i != 99:
                out_fh.write(line)
            else:
                out_fh.write(f"5 cz {radius}")
```

* `mcnp_pstudy.pl` would be a better tool here

### The Regular Expression Basher

``` python
import re

def change_mat_number(old_number, new_number):
     with open("old_victim.imcnp") as in_fh, open(f"new_victim.imcnp") as out_fh:
         for line in in_fh:
             if is_blue_moon_and_friday_13(line):
                 out_fh.write(re.sub(r"m(\d+)/////# how many backslaches???", f"m{new_number}"))
             elif is_dec_12_2012(line):
                 pass # so much more abuse
             else:
                 out_fh.write(line)
```

# Oh but, my script is different...
## Does it use context-free parsing in addition to regular expressions?

### If not: Then it is provably non-robust
* Regular expressions are *proven* to not be able parse:
    * Parantheses groupings
    * Operator Precedence <sup>2</sup>

<sup>2</sup>K. D. Cooper and L. Torczon, Engineering a Compiler. San Francisco, CA: Morgan Kaufmann Publishers, 2004.


# Why MontePy
* Takes minutia out of the automation
    * Does not require pre-templating
* Implements Context-Free parsing
     * Can preserve user formatting
     * Understands to order of operations for CSG
* Allows you to focus on semantics; not syntax!

In [ ]:
import montepy

problem = montepy.read_input("models/pin_cell.imcnp")

importances = {1: 0.005, 2: 0.1, 3: 1.0, 5: 1.235}
for cell_num, importance in importances.items():
    problem.cells[cell_num].importance.neutron = importance
    print(problem.cells[cell_num].mcnp_str())

In [ ]:
# create a universe and fill another cell with it (hide)
universe = montepy.Universe(123)
problem.universes.append(universe)
# Slicing is inclusive, so this would include all cells with numbers between 1 and 5.
universe.claim(problem.cells[1:5])
print(problem.cells[1:5])

# Status of support


## Note: All support Based on Public MCNP User Manual (6.2, 6.3.0, 6.3.1)
* We Are aware MCNP has many undocumented "features", and will tolerate a lot of syntax abuse
    * MontePy will only ever support officially documented syntax

## Great fully Object-Oriented Support
   * Cells
   * Materials
   * Universes
   * cylindrical, and planar surfaces (e.g., `CZ`, `PZ`)
      * has `plane.location` and `cylinder.radius` properties


## Partial Object-Oriented Support
   * Other surfaces
        * Does not have meaningful properties for surface coefficients
        * `surf.surface_constants` instead of `cylinder.radius`

## Limited Object-Oriented Support
   * Almost all Other Inputs
   * Can read information
   * Determined user can change input values if you know the location in the list
   * Has only properties for the classifier mnemonic


## Lacking any meaningful support
* Special complications
   * `DE`
     * it completely changes the syntax around `log`, will only be parroted.
## Unsupported Syntaxes
 * Vertical mode
 * Cell `Like n but...`
 * Will raise `UnsupportedFeatureException`

# Code Quality 
* Follows Modern Software Practices
    * Requires Peer reviews on Pull Requests
    * Uses Continuous Integration/Deployment (22 checks)
* Has extensive test suite
    * over 800 tests
       * Tests are ran against 5 versions of pythons (3.9-3.13)
       * With two versions for both dependencies (numpy 1, 2 and sly 0.4, 0.5)
    * ~ 98% code coverage
* Stable versioning system.
   * Uses semantic versioning (`Major`, `Minor`, `Patch`). Only `Major` Releases should break old code.
   * Releases are immutable on [PyPI](https://pypi.org/project/montepy/) and [conda-forge](https://anaconda.org/conda-forge/montepy).
  * `pip install montepy`
  * `conda install conda-forge::montepy`

# MontePy Website
* It's on GitHub!
    * [github.com/idaholab/MontePy](https://github.com/idaholab/MontePy)
    * Go here for filing issues:
         * bugs
         * Feature requests
         * typos, etc
    * Star us!
    * Contribute here




## Documentation is at [montepy.org](https://www.montepy.org/en/stable/)


In [ ]:
from _config import IFrame

IFrame("https://www.montepy.org/en/stable/")

# Explain Top level menus

# Accessing Tutorials
* Use the getting started guide

In [ ]:
IFrame("https://www.montepy.org/en/stable/starting.html#specifying-nuclides")

# Accessing Doc Strings
## Structure
* Top Level
    * `MCNP_Problem`
    * `Cell`
    * Collections like: `Cells`, `Surfaces`, `Materials`, etc.
    * Internal abstract classes `MCNP_Object`, etc.
* `montepy.data_inputs`
   * Everything that *may* show up in the data block
   * Includes cell modifiers `imp`, `vol`, `fill`, etc.
* `montepy.input_parser`
   * these are not the droids you are looking for
* `montepy.surfaces`
   * all surface classes

# Finding Cell Doc-strings
new sub-slide

In [ ]:
IFrame("https://www.montepy.org/en/v1.0.0a1/api/modules.html")

* Click cell
* Discuss methods versus properties
* Click on `geometry`
* Discuss the types

# Finding the Material Doc-strings

In [ ]:
IFrame("https://www.montepy.org/en/v1.0.0a1/api/modules.html")

* click on material
* Discuss top level class and data
* click `Material`
* Click on `add_nuclide`
* discuss doc-string contract
* discuss doc-string type hints
* click on `append`
* discuss `tuple[]` notation

# Note on Teaching Style
* We do not have time to make you experts
* Goal: teach self-efficacy
   * Introduce to MontePy organization dogma
   * Where to look for resources
   * How to approach software defined modeling

# Acknowledgments 

Work supported through the Advanced Fuels Campaign (AFC) under DOE Idaho Operations Office Contract DE-AC07-05ID14517. The authors wish to thank the U.S. Department of Energy Office of Isotope R&D and Production for their vital and continued support and funding of the Co-60 program at INL under Contract No. DE-AC07-05ID14517. Co-60 is sold by the National Isotope Development Center (NIDC). Quotes on Co-60 can be obtained from NIDC at www.isotopes.gov/products/cobalt. This research made use of Idaho National Laboratory's High Performance Computing systems located at the Collaborative Computing Center and supported by the Office of Nuclear Energy of the U.S. Department of Energy and the Nuclear Science User Facilities under Contract No. DE-AC07-05ID14517.

# Questions ?

## You May also Enjoy:

* "A Practical Guide to Parsing MCNP Inputs: Lessons Learned from Implementing Context-Free Parsing in MontePy"
    * Tuesday, 10:25 - 10:50 AM
    * Lawrence B

## Contact Us
* Documentation: [montepy.org](https://www.montepy.org/en/stable/)
* repository: [github.com/idaholab/MontePy](https://github.com/idaholab/MontePy)
* Lead Developer: [Micah.Gale@INL.gov](mailto:micah.gale@inl.gov)
  
![QR code for montepy.org](figs/montepy_qr.png)

# Is MontePy NQA-1?

## No

# Is MCNP NQA-1?

## No

* Per [LA-UR-20-26666](https://mcnp.lanl.gov/pdf_files/TechReport_2020_LANL_XCP-320-008LA-UR-20-26666_RisingBrownEtAl.pdf)

> The general purpose MCNP6 particle transport code can be used for many applications ...  It would unreasonable for the code development team to develop the code under the strict ASME NQA-1-2008 (NQA-1) Quality Assurance Requirements for Nuclear Facility Applications documented standards for all of these application areas ... For other users and institutions, the requirements may vary based on how each institution chooses to follow the NQA-1 and other international standards.


* It is impossible under NQA-1 to have "off-the-shelf" NQA-1 products
   * Must complete commercial grade dedication, or
   * Add vendor to a Qaulified Supplier List, which requires extended back and forth with vendor, and extensive auditing.


# Should You Qualify MontePy

## Probably not: do it for your end application.